In [1]:
import pandas as pd
import os 
import csv
import re


In [2]:
#reading in bacteria list

with open('bac_list.csv') as f:
    reader = csv.reader(f)
    blist = list(reader)

bacteria = []

for bac in blist[2:]:
    if len(bac) > 0:
         bacteria += bac[0].split(' ')

In [3]:
# fxn to truncate at references section

def refclean(doc):
    
    citepat1 = re.compile('(1\.\s [A-Z].*\n.*\n)')
    citepat2 = re.compile('(\nREFERENCES\n)', re.IGNORECASE)

    temp = citepat2.split(doc, maxsplit=1)

    if len(temp) > 1:
        cleaned = temp[0]
        
    else:
        temp2 = citepat1.split(doc)

        if len(temp2) > 1:
            if len(temp2) > 2:
                cleaned = ' '.join(temp[:-1])
            else:
                cleaned = temp2[0]
        else:
            cleaned = temp2
            
    return cleaned

#fxn to remove urls



def clean(doc):

    cleaned = re.sub(r"http\S+", "", doc)
    cleaned = re.sub(r"www\S+", "", doc)
    cleaned = re.sub(r"\((fig|figure|table)\..*\)", "", cleaned, flags=re.IGNORECASE)
    cleaned = re.sub(r"(\([0-9]*[0-9],*.*\).?)","",cleaned)
    cleaned = re.sub(r"(\[[0-9]*[0-9],*.*\].?)","",cleaned)
    
    return cleaned
    


# fxn to break into paragraphs

def paras(doc): 
    
    paras = doc.split('\n\n')
    #paras = doc.split('.\n\n')
    paras = [para for para in paras if (len(para) > 30)]
    
    topara = []
    
    for para in paras:
        topara.append(re.sub(r'\n\n', " ", para)) 
    
    return topara
    
    

# fxn to only keep paragraphs w/terms appearing in bacteria list
    
def bacfilter(paras, blist):
    
    tokeep = []
    
    for para in paras:
        temp = para.lower()
        if any(check in temp.split(' ') for check in blist):
            tokeep.append(para)
    
    return tokeep

In [4]:
# Reading in and processing raw .txt files then writing filtered & cleaned to file
txtfiles = os.listdir("TxtData")


for txtfile in txtfiles:
    
    with open("TxtData/"+ txtfile, "r") as f:
        doc = f.read()
    
    #clean1 = refclean(str(doc))
    clean2 = clean(str(doc))
           
    topara = paras(clean2)
    
    filtered = bacfilter(topara, bacteria)
    
    paranum = str(len(filtered))
    
    filtdoc = paranum + "_PARAGRAPHS\n\n" + ".\n\n".join(filtered)
    
    path = "Filtered1/" + txtfile[:-4] + "_FILTERED.txt"
    
    if len(filtdoc) > 20:
        with open(path, "w") as f:
            f.write(filtdoc)


In [ ]:
filtered